# Example of using Delta Lake 0.4 without Databricks

[Delta Lake](https://delta.io/) is an open-source storage layer that brings ACID transactions to Apache Spark™ and big data workloads. But it is **way more** then that!

This is a rewritten notebook example from this [blog post](https://databricks.com/blog/2019/10/03/simple-reliable-upserts-and-deletes-on-delta-lake-tables-using-python-apis.html) by Databricks. The intension is to show why Delta Lake is a big deal and how to run Delta Lake without a Databricks services.

Delta Lake examples in this notebook:
* Convert data to as Delta Lake format
* Create Delta Lake table
* Spark SQL capabilities
* Delete data
* Update data
* View audit history of table
* Merge (union) of two tables which remove duplicates, updates rows and add a new row

For testing this docker can be used: ```docker run -it --rm -p 8888:8888 -p 4040:4040 jupyter/pyspark-notebook```

### Author
Anders Boje Larsen - [alarsen@deloitte.dk](alarsen@deloitte.dk) - [LinkedIn](https://www.linkedin.com/in/andersboje/)

### Data Preparation
Configure locations for the source file and where the Delta Lake Table will be stored

In [29]:
tripdelaysFilePath = "departuredelays.csv" 
pathToEventsTable = "departureDelays.delta"
flightdata = "https://raw.githubusercontent.com/drabastomek/learningPySpark/master/Chapter03/flight-data/departuredelays.csv"

In [30]:
!pip install --upgrade pyspark
!pip install pandas

Requirement already up-to-date: pyspark in /Users/alarsen/miniconda3/lib/python3.7/site-packages (2.4.4)


In [31]:
#Download data flight dataset
!rm -fr departureDelays.delta
!wget https://raw.githubusercontent.com/drabastomek/learningPySpark/master/Chapter03/flight-data/departuredelays.csv

--2019-10-24 21:17:48--  https://raw.githubusercontent.com/drabastomek/learningPySpark/master/Chapter03/flight-data/departuredelays.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)...199.232.40.133
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|199.232.40.133|:443...connected.
HTTP request sent, awaiting response...200 OK
Length: 33396236 (32M) [text/plain]
Saving to: ‘departuredelays.csv.5’

departuredelays.csv 100%[===================>]  31.85M  9.27MB/s    in 3.5s    

2019-10-24 21:17:54 (9.10 MB/s) - ‘departuredelays.csv.5’ saved [33396236/33396236]



In [32]:
import pandas as pd
from pyspark.sql import SQLContext, SparkSession
from pyspark import SparkContext, SparkConf
import os

os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages io.delta:delta-core_2.11:0.4.0 pyspark-shell'

sc_conf = SparkConf()
sc_conf.set('spark.databricks.delta.retentionDurationCheck.enabled', 'false')
sc_conf.set('spark.sql.extensions', 'io.delta.sql.DeltaSparkSessionExtension')

try:
    sc.stop()
    sc = SparkContext(conf=sc_conf)
except:
    sc = SparkContext(conf=sc_conf)

spark = SparkSession(sc)

Create `departureDelays` DataFrame

In [33]:
departureDelays = spark.read.option("header", "true").option("inferSchema", "true").csv(tripdelaysFilePath)

Save table as Delta Lake (update `pathToEventsTable` to match the following location

In [34]:
departureDelays.write.format("delta").mode("overwrite").save(pathToEventsTable)

Load Delta Lake table

In [35]:
delays_delta = spark.read.format("delta").load(pathToEventsTable)
delays_delta.createOrReplaceTempView("delays_delta")

Get count of rows

In [36]:
spark.sql("select count(1) from delays_delta where origin = 'SEA' and destination = 'SFO'").toPandas()

,count(1)
0,1698


**Review File System**: Note there are four files initially created as part of the table creation.

In [37]:
%ls $pathToEventsTable

_delta_log/
part-00000-17468cb5-af06-4072-8a84-8a394ecfd7db-c000.snappy.parquet
part-00001-de10ca1b-7c5a-4e7b-9f54-b82583083a04-c000.snappy.parquet
part-00002-8e8b25b0-4a3d-4ed8-aec1-1d64e51dec1e-c000.snappy.parquet
part-00003-a4556043-2fbf-436a-9f10-92d4f86b666f-c000.snappy.parquet
part-00004-47f3ed39-4806-400d-8990-db6a3887f171-c000.snappy.parquet
part-00005-368b37ed-02cf-45f6-9d53-1f2e41041cd5-c000.snappy.parquet
part-00006-3c521f83-7f0f-41ae-b6c4-b7306813eec3-c000.snappy.parquet
part-00007-0c346081-4a64-44f8-9368-44d33d126348-c000.snappy.parquet


### Deletes
With Delta Lake, you can delete data with the Python API

In [38]:
from delta.tables import *
from pyspark.sql.functions import *
deltaTable = DeltaTable.forPath(spark, pathToEventsTable)
deltaTable.delete("delay < 0") 

In [39]:
# Get Row Count
spark.sql("select count(1) from delays_delta where origin = 'SEA' and destination = 'SFO'").toPandas()

,count(1)
0,837


**Review File System**: Note that while we deleted early (and on-time) flights, there are now eight files (instead of the four files initially created as part of the table creation).

In [40]:
%ls $pathToEventsTable

_delta_log/
part-00000-17468cb5-af06-4072-8a84-8a394ecfd7db-c000.snappy.parquet
part-00000-b0252edb-5ab7-45f4-abe3-f86c61e1bb82-c000.snappy.parquet
part-00001-828c7e25-402b-4ba8-ba0c-3e96c5499a6c-c000.snappy.parquet
part-00001-de10ca1b-7c5a-4e7b-9f54-b82583083a04-c000.snappy.parquet
part-00002-239c1673-bab9-45df-9400-7ccee3f09ae7-c000.snappy.parquet
part-00002-8e8b25b0-4a3d-4ed8-aec1-1d64e51dec1e-c000.snappy.parquet
part-00003-a4556043-2fbf-436a-9f10-92d4f86b666f-c000.snappy.parquet
part-00003-afec5f76-09ad-4439-bf8f-7adcd52bb09c-c000.snappy.parquet
part-00004-47f3ed39-4806-400d-8990-db6a3887f171-c000.snappy.parquet
part-00004-d90ca5bb-b673-4cdc-aa1b-e5402fd20b39-c000.snappy.parquet
part-00005-368b37ed-02cf-45f6-9d53-1f2e41041cd5-c000.snappy.parquet
part-00005-a9d0a35f-5af8-4365-82a1-a47d42c13909-c000.snappy.parquet
part-00006-2a90baca-f0b9-42d6-bb9d-3d8e7e939502-c000.snappy.parquet
part-00006-3c521f83-7f0f-41ae-b6c4-b7306813eec3-c000.snappy.parquet
part-00007-0c346081-4a64-44f8-9368-4

### Updates
Update flights originating from Detroit (DTW) to now be from Seattle (SEA)

In [41]:
deltaTable.update("origin = 'DTW'", { "origin": "'SEA'" } ) 

In [42]:
spark.sql("select count(1) from delays_delta where origin = 'SEA' and destination = 'SFO'").toPandas()

,count(1)
0,986


### View History
View the table history (note the create table, insert, and update operations)

In [43]:
deltaTable.history().toPandas()

,version,timestamp,userId,userName,operation,operationParameters,job,notebook,clusterId,readVersion,isolationLevel,isBlindAppend
0,2,2019-10-24 21:18:08,None,None,UPDATE,{'predicate': '(origin#1503 = DTW)'},None,None,None,1.0,None,False
1,1,2019-10-24 21:18:03,None,None,DELETE,"{'predicate': '[""(`delay` < 0)""]'}",None,None,None,0.0,None,False
2,0,2019-10-24 21:17:58,None,None,WRITE,"{'mode': 'Overwrite', 'partitionBy': '[]'}",None,None,None,NaN,None,False


Calculate counts for each version of the table

In [44]:
dfv0 = spark.read.format("delta").option("versionAsOf", 0).load("departureDelays.delta")
dfv1 = spark.read.format("delta").option("versionAsOf", 1).load("departureDelays.delta")
dfv2 = spark.read.format("delta").option("versionAsOf", 2).load("departureDelays.delta")

cnt0 = dfv0.where("origin = 'SEA'").where("destination = 'SFO'").count()
cnt1 = dfv1.where("origin = 'SEA'").where("destination = 'SFO'").count()
cnt2 = dfv2.where("origin = 'SEA'").where("destination = 'SFO'").count()

print("SEA -> SFO Counts: Create Table: %s, Delete: %s, Update: %s" % (cnt0, cnt1, cnt2))

SEA -> SFO Counts: Create Table: 1698, Delete: 837, Update: 986


**Review File System**: Note the number of files based on the preceding operations.

In [45]:
%ls $pathToEventsTable

_delta_log/
part-00000-149d02e6-b092-468e-8dc7-e09a0dcb5623-c000.snappy.parquet
part-00000-17468cb5-af06-4072-8a84-8a394ecfd7db-c000.snappy.parquet
part-00000-b0252edb-5ab7-45f4-abe3-f86c61e1bb82-c000.snappy.parquet
part-00001-5c1c47e0-712d-4bbc-bfc4-85b3cc7339df-c000.snappy.parquet
part-00001-828c7e25-402b-4ba8-ba0c-3e96c5499a6c-c000.snappy.parquet
part-00001-de10ca1b-7c5a-4e7b-9f54-b82583083a04-c000.snappy.parquet
part-00002-239c1673-bab9-45df-9400-7ccee3f09ae7-c000.snappy.parquet
part-00002-8e8b25b0-4a3d-4ed8-aec1-1d64e51dec1e-c000.snappy.parquet
part-00002-99bc922b-eb2f-42bb-b156-6cecffd12be9-c000.snappy.parquet
part-00003-a4556043-2fbf-436a-9f10-92d4f86b666f-c000.snappy.parquet
part-00003-afec5f76-09ad-4439-bf8f-7adcd52bb09c-c000.snappy.parquet
part-00004-47f3ed39-4806-400d-8990-db6a3887f171-c000.snappy.parquet
part-00004-d90ca5bb-b673-4cdc-aa1b-e5402fd20b39-c000.snappy.parquet
part-00005-368b37ed-02cf-45f6-9d53-1f2e41041cd5-c000.snappy.parquet
part-00005-a9d0a35f-5af8-4365-82a1-a

### Vacuum
Remove older data (by default 7 days) 

In [46]:
deltaTable.vacuum(0)

DataFrame[]

In [47]:
%ls $pathToEventsTable

_delta_log/
part-00000-149d02e6-b092-468e-8dc7-e09a0dcb5623-c000.snappy.parquet
part-00000-b0252edb-5ab7-45f4-abe3-f86c61e1bb82-c000.snappy.parquet
part-00001-5c1c47e0-712d-4bbc-bfc4-85b3cc7339df-c000.snappy.parquet
part-00001-828c7e25-402b-4ba8-ba0c-3e96c5499a6c-c000.snappy.parquet
part-00002-99bc922b-eb2f-42bb-b156-6cecffd12be9-c000.snappy.parquet
part-00003-afec5f76-09ad-4439-bf8f-7adcd52bb09c-c000.snappy.parquet
part-00006-2a90baca-f0b9-42d6-bb9d-3d8e7e939502-c000.snappy.parquet
part-00007-fed7d610-246e-4f66-a404-2198e862d3b0-c000.snappy.parquet


And let's not forget, Delta Lake 0.4.0 also includes `MERGE` in the Python API!

### Merge
Let's merge another table with the `departureDelays` table with [data deduplication](https://docs.delta.io/0.4.0/delta-update.html#data-deduplication-when-writing-into-delta-tables).  Let's start by viewing data that will be impacted by the merge.

In [48]:
spark.sql("select * from delays_delta where origin = 'SEA' and destination = 'SFO' and date like '1010%' order by date limit 10").toPandas()

,date,delay,distance,origin,destination
0,1010521,0,590,SEA,SFO
1,1010710,31,590,SEA,SFO
2,1010730,5,590,SEA,SFO
3,1010955,104,590,SEA,SFO


Next, let's create our `merge_table` which contains three rows:
* 1010710: this row is a duplicate
* 1010521: this row will be updated with a new delay value
* 1010822: this is a new row

In [49]:
items = [(1010521, 10, 590, 'SEA', 'SFO'), (1010710, 31, 590, 'SEA', 'SFO'), (1010832, 31, 590, 'SEA', 'SFO')]
cols = ['date', 'delay', 'distance', 'origin', 'destination']
merge_table = spark.createDataFrame(items, cols)
merge_table.toPandas()

,date,delay,distance,origin,destination
0,1010521,10,590,SEA,SFO
1,1010710,31,590,SEA,SFO
2,1010832,31,590,SEA,SFO


Let's run our merge statement that will handle the duplicates, updates, and add a new row

In [50]:
deltaTable.alias("flights") \
    .merge(merge_table.alias("updates"),"flights.date = updates.date") \
    .whenMatchedUpdate(set = { "delay" : "updates.delay" } ) \
    .whenNotMatchedInsertAll() \
    .execute()

In [51]:
spark.sql("select * from delays_delta where origin = 'SEA' and destination = 'SFO' and date like '1010%' order by date limit 10").toPandas()

,date,delay,distance,origin,destination
0,1010521,10,590,SEA,SFO
1,1010710,31,590,SEA,SFO
2,1010730,5,590,SEA,SFO
3,1010832,31,590,SEA,SFO
4,1010955,104,590,SEA,SFO


As noted in the previous cells, notice the following:
* There is only one row for the date `1010710` as `merge` automatically takes care of **data deduplication**
* The row for the date `1010521` has the `delay` value **updated** from 0 to 10.
* The row for the date `1010832` has been added as this date did not exist, hence it was **inserted**.